In [2]:
import datetime as dt
from utility import (
    read_sql,
    fd_alive_funds,
    fd_basicinfo,
    fd_typeclass,
    fd_hshkiport,
    fd_assetportfolio,
    fd_derieden,
    nearest_report_date
)
import pandas as pd
from PyFin.api import makeSchedule
from PyFin.api import BizDayConventions

# 1. 基金分类
-----------------------

In [76]:
# 获取回溯的报告日

today = dt.datetime.today()
date_3yrs_ago = today.replace(year=today.year - 3)
report_dates_begin = nearest_report_date(date_3yrs_ago.strftime("%Y%m%d"))
current_date = today.strftime("%Y%m%d")

In [79]:
# 获取所有目标股票型基金（清算结束日期晚于当前日）

security_ids = fd_alive_funds(current_date, 1)

In [80]:
basic_info = fd_basicinfo(security_ids, current_date)
type_info = fd_typeclass(security_ids, current_date)
hkport_info = fd_hshkiport(security_ids, report_dates_begin, current_date)
hkport_info = hkport_info[hkport_info.INDUSTRYNAME == "合计"].groupby(["SECURITYID"], as_index=False)[["ACCNETMKTCAP"]].mean()
asset_port = fd_assetportfolio(security_ids, report_dates_begin, current_date).groupby("SECURITYID", as_index=False)[["EQUITYINVERTO"]].mean()

In [81]:
df = pd.merge(basic_info, type_info, on="SECURITYID", how="inner")
df = pd.merge(df, asset_port, on="SECURITYID", how="left")
df = pd.merge(df, hkport_info, on="SECURITYID", how="left").drop_duplicates()

if "ACCNETMKTCAP" in df:
    df["ACCNETMKTCAP"] = df["ACCNETMKTCAP"].fillna(0)
else:
    df["ACCNETMKTCAP"] = 0.0

In [82]:
# 分类

flag = df["ACCNETMKTCAP"] / df["EQUITYINVERTO"] > 0.5
df.loc[flag, "股票型（子类）"] = "港股通股票型基金"

df.loc[(~flag) & (df["L3NAME"].isin(["其他行业股票型基金", "偏股型基金（股票上下限60%-95%）", '标准股票型基金', '医药行业股票型基金', '封闭式标准股票型基金'])), "股票型（子类）"] = "普通股票型基金"
df.loc[(~flag) & (df["L3NAME"].isin(["标准指数股票型基金"])), "股票型（子类）"] = "被动指数型基金"
df.loc[(~flag) & (df["L3NAME"].isin(["增强指数股票型基金"])), "股票型（子类）"] = "增强指数型基金"

In [83]:
df.groupby("股票型（子类）").count()

,SECURITYID,FDNAME,SNAMECOMP,FSYMBOL,FDNATURE,INVESTSTYLE,L1CODE,L1NAME,L2CODE,L2NAME,L3CODE,L3NAME,EQUITYINVERTO,ACCNETMKTCAP
股票型（子类）,,,,,,,,,,,,,,
增强指数型基金,63,63,45,63,63,63,63,63,63,63,63,63,60,63
普通股票型基金,205,205,59,205,205,205,205,205,205,205,205,205,204,205
港股通股票型基金,12,12,8,12,12,12,12,12,12,12,12,12,12,12
被动指数型基金,328,328,202,328,328,328,328,328,328,328,328,328,315,328


In [84]:
df.sort_values("SECURITYID")[:20]

,SECURITYID,FDNAME,SNAMECOMP,FSYMBOL,FDNATURE,INVESTSTYLE,L1CODE,L1NAME,L2CODE,L2NAME,L3CODE,L3NAME,EQUITYINVERTO,ACCNETMKTCAP,股票型（子类）
0,1030000030,国泰沪深300指数证券投资基金,国泰沪深300指数,020011,证券投资基金,指数型,1,股票基金,1.2,指数股票型基金,1.2.1,标准指数股票型基金,93.975000,0.0,被动指数型基金
1,1030000034,华安MSCI中国A股指数增强型证券投资基金,None,040002,证券投资基金,指数型,1,股票基金,1.2,指数股票型基金,1.2.2,增强指数股票型基金,94.049167,0.0,增强指数型基金
2,1030000045,博时裕富沪深300指数证券投资基金,博时沪深300指数,050002,证券投资基金,指数型,1,股票基金,1.2,指数股票型基金,1.2.1,标准指数股票型基金,93.689167,0.0,被动指数型基金
3,1030000096,易方达上证50指数增强型证券投资基金,易方达上证50增强,110003,证券投资基金,指数型,1,股票基金,1.2,指数股票型基金,1.2.2,增强指数股票型基金,92.823333,0.0,增强指数型基金
554,1030000136,融通深证100指数证券投资基金,融通深证100指数,161604,证券投资基金,指数型,1,股票基金,1.2,指数股票型基金,1.2.2,增强指数股票型基金,94.824167,0.0,增强指数型基金
580,1030000149,泰达宏利首选企业股票型证券投资基金,None,162208,证券投资基金,增值型,1,股票基金,1.1,标准股票型基金,1.1.1,标准股票型基金,93.647500,0.0,普通股票型基金
564,1030000159,银华-道琼斯88精选证券投资基金,None,180003,证券投资基金,指数型,1,股票基金,1.2,指数股票型基金,1.2.2,增强指数股票型基金,88.533333,0.0,增强指数型基金
577,1030000168,长城久泰沪深300指数证券投资基金,长城久泰沪深300指数,200002,证券投资基金,指数型,1,股票基金,1.2,指数股票型基金,1.2.2,增强指数股票型基金,94.545000,0.0,增强指数型基金
720,1030000245,申万菱信沪深300指数增强型证券投资基金,申万菱信沪深300指数增强,310318,证券投资基金,指数型,1,股票基金,1.2,指数股票型基金,1.2.2,增强指数股票型基金,91.097500,0.0,增强指数型基金
649,1030000258,兴全全球视野股票型证券投资基金,None,340006,证券投资基金,成长型,1,股票基金,1.1,标准股票型基金,1.1.1,标准股票型基金,89.563333,0.0,普通股票型基金


## 1.1 Put it all together

In [85]:
def create_stk_fund_info(trade_dt):
    trade_dt = dt.datetime.strptime(trade_dt, "%Y%m%d")
    date_3yrs_ago = trade_dt.replace(year=trade_dt.year - 3)
    report_dates_begin = nearest_report_date(date_3yrs_ago.strftime("%Y%m%d"))
    current_date = trade_dt.strftime("%Y%m%d")
    
    security_ids = fd_alive_funds(current_date, 1)
    basic_info = fd_basicinfo(security_ids, current_date)
    type_info = fd_typeclass(security_ids, current_date)
    hkport_info = fd_hshkiport(security_ids, report_dates_begin, current_date)
    hkport_info = hkport_info[hkport_info.INDUSTRYNAME == "合计"].groupby(["SECURITYID"], as_index=False)[["ACCNETMKTCAP"]].mean()
    asset_port = fd_assetportfolio(security_ids, report_dates_begin, current_date).groupby("SECURITYID", as_index=False)[["EQUITYINVERTO"]].mean()
    
    df = pd.merge(basic_info, type_info, on="SECURITYID", how="inner")
    df = pd.merge(df, asset_port, on="SECURITYID", how="left")
    df = pd.merge(df, hkport_info, on="SECURITYID", how="left").drop_duplicates()
    if "ACCNETMKTCAP" in df:
        df["ACCNETMKTCAP"] = df["ACCNETMKTCAP"].fillna(0)
    else:
        df["ACCNETMKTCAP"] = 0.0
    
    flag = df["ACCNETMKTCAP"] / df["EQUITYINVERTO"] > 0.5
    df.loc[flag, "股票型（子类）"] = "港股通股票型基金"

    df.loc[(~flag) & (df["L3NAME"].isin(["其他行业股票型基金", "偏股型基金（股票上下限60%-95%）", '标准股票型基金', '医药行业股票型基金', '封闭式标准股票型基金'])), "股票型（子类）"] = "普通股票型基金"
    df.loc[(~flag) & (df["L3NAME"].isin(["标准指数股票型基金"])), "股票型（子类）"] = "被动指数型基金"
    df.loc[(~flag) & (df["L3NAME"].isin(["增强指数股票型基金"])), "股票型（子类）"] = "增强指数型基金"
    
    return df.sort_values("SECURITYID").dropna(subset=["股票型（子类）"])

In [86]:
%%time

create_stk_fund_info("20211109").groupby("股票型（子类）").count()

Wall time: 3.78 s


,SECURITYID,FDNAME,SNAMECOMP,FSYMBOL,FDNATURE,INVESTSTYLE,L1CODE,L1NAME,L2CODE,L2NAME,L3CODE,L3NAME,EQUITYINVERTO,ACCNETMKTCAP
股票型（子类）,,,,,,,,,,,,,,
增强指数型基金,63,63,45,63,63,63,63,63,63,63,63,63,60,63
普通股票型基金,205,205,59,205,205,205,205,205,205,205,205,205,204,205
港股通股票型基金,12,12,8,12,12,12,12,12,12,12,12,12,12,12
被动指数型基金,328,328,202,328,328,328,328,328,328,328,328,328,315,328


# 2. 基金指数
-----------------

In [87]:
# 我们只计算指数的每日收益
# 在确定指数的基准日之后，可以直接使用收益计算指数的值。
# 使用 TQ_FD_DERIVEDN 获取基金的净值情况

start_dt = "2015-01-31"
final_trade_dt = "2021-11-09"
nav_type = "REPAIRUNITNAV"

rebalance_dates = [d.strftime("%Y%m%d") for d in makeSchedule(start_dt, final_trade_dt, tenor="3M", calendar="china.sse", dateRule=BizDayConventions.ModifiedFollowing)]

In [88]:
dfs = []

for i, trade_dt in enumerate(rebalance_dates[1:]):
    pre_trade_dt = rebalance_dates[i]
    print(pre_trade_dt, trade_dt)
    fund_info = create_stk_fund_info(trade_dt)
    pre_nav_info = fd_derieden(fund_info.SECURITYID.tolist(), pre_trade_dt).rename(columns={nav_type: "PRE" + nav_type})
    nav_info = fd_derieden(fund_info.SECURITYID.tolist(), trade_dt)

    total_df = pd.merge(fund_info, nav_info, on=["SECURITYID"])
    total_df = pd.merge(total_df, pre_nav_info, on=["SECURITYID"])
    total_df["chg."] = total_df[nav_type] / total_df["PRE" + nav_type] - 1.0
    res = total_df.groupby("股票型（子类）")[["chg."]].mean()
    dfs.append(res)

20150130 20150430
20150430 20150731
20150731 20151030
20151030 20160129
20160129 20160429
20160429 20160729
20160729 20161031
20161031 20170126
20170126 20170428
20170428 20170731
20170731 20171031
20171031 20180131
20180131 20180427
20180427 20180731
20180731 20181031
20181031 20190131
20190131 20190430
20190430 20190731
20190731 20191031
20191031 20200123
20200123 20200430
20200430 20200731
20200731 20201030
20201030 20210129
20210129 20210430
20210430 20210730
20210730 20211029
20211029 20211109


In [89]:
final_report = pd.concat(dfs, keys=rebalance_dates[1:]).reset_index()
final_report.pivot_table(index="level_0", columns="股票型（子类）", values="chg.").to_excel("010_股票型基金_bak.xlsx")

In [90]:
final_report.pivot_table(index="level_0", columns="股票型（子类）", values="chg.")

股票型（子类）,增强指数型基金,普通股票型基金,港股通股票型基金,被动指数型基金
level_0,,,,
20150430,0.373081,0.445309,NaN,0.399033
20150731,-0.150522,-0.127085,NaN,-0.159062
20151030,-0.062868,0.000142,NaN,-0.064587
20160129,-0.154214,-0.172297,NaN,-0.177011
20160429,0.088368,0.101380,NaN,0.084537
20160729,0.047092,0.055730,0.000000,0.037961
20161031,0.041936,0.035465,0.033208,0.036679
20170126,0.000413,-0.035175,0.032578,-0.015172
20170428,0.017795,0.033252,0.079169,0.005582
